# （一）统计显著性检验
2.1.2 AUC的统计不确定性（样本量有限导致的随机波动）


2.1.8  ROC曲线的不确定性带（置信区间)

In [ ]:
print("\n=== Statistical Significance Analysis ===")

from sklearn.utils import resample
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt

def bootstrap_auc(y_true, y_scores, n_bootstrap=1000, confidence_level=0.95):
    """
    使用Bootstrap方法计算AUC的置信区间
    
    参数:
    - y_true: 真实标签
    - y_scores: 预测分数
    - n_bootstrap: Bootstrap采样次数
    - confidence_level: 置信水平（默认95%）
    
    返回:
    - auc_mean: AUC均值
    - auc_ci_lower: 置信区间下界
    - auc_ci_upper: 置信区间上界
    - auc_std: AUC标准差
    - bootstrap_aucs: 所有Bootstrap样本的AUC值
    """
    bootstrap_aucs = []
    n_samples = len(y_true)
    
    print(f"  Performing {n_bootstrap} bootstrap iterations...")
    for i in range(n_bootstrap):
        # 有放回地随机抽样
        indices = resample(range(n_samples), n_samples=n_samples, random_state=i)
        y_true_boot = y_true[indices]
        y_scores_boot = y_scores[indices]
        
        # 计算该样本的AUC
        fpr_boot, tpr_boot, _ = roc_curve(y_true_boot, y_scores_boot)
        auc_boot = auc(fpr_boot, tpr_boot)
        bootstrap_aucs.append(auc_boot)
        
        if (i + 1) % 200 == 0:
            print(f"  Progress: {i+1}/{n_bootstrap} iterations completed")
    
    bootstrap_aucs = np.array(bootstrap_aucs)
    auc_mean = np.mean(bootstrap_aucs)
    auc_std = np.std(bootstrap_aucs)
    
    # 计算置信区间（使用百分位数方法）
    alpha = 1 - confidence_level
    auc_ci_lower = np.percentile(bootstrap_aucs, 100 * alpha / 2)
    auc_ci_upper = np.percentile(bootstrap_aucs, 100 * (1 - alpha / 2))
    
    return auc_mean, auc_ci_lower, auc_ci_upper, auc_std, bootstrap_aucs


def bootstrap_roc_curve(y_true, y_scores, n_bootstrap=1000, confidence_level=0.95):
    """
    使用Bootstrap方法计算ROC曲线的置信区间
    
    返回:
    - mean_fpr: 统一的FPR网格
    - mean_tpr: 平均TPR
    - tpr_ci_lower: TPR置信区间下界
    - tpr_ci_upper: TPR置信区间上界
    """
    print(f"  Computing ROC curve confidence bands with {n_bootstrap} bootstrap samples...")
    
    # 创建统一的FPR网格（0到1之间均匀分布）
    mean_fpr = np.linspace(0, 1, 100)
    tprs = []
    
    n_samples = len(y_true)
    
    for i in range(n_bootstrap):
        # Bootstrap采样
        indices = resample(range(n_samples), n_samples=n_samples, random_state=i)
        y_true_boot = y_true[indices]
        y_scores_boot = y_scores[indices]
        
        # 计算ROC曲线
        fpr_boot, tpr_boot, _ = roc_curve(y_true_boot, y_scores_boot)
        
        # 插值到统一的FPR网格
        tpr_interp = np.interp(mean_fpr, fpr_boot, tpr_boot)
        tpr_interp[0] = 0.0  # 确保起点是(0,0)
        tprs.append(tpr_interp)
        
        if (i + 1) % 200 == 0:
            print(f"  Progress: {i+1}/{n_bootstrap} ROC curves computed")
    
    tprs = np.array(tprs)
    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0  # 确保终点是(1,1)
    
    # 计算置信区间
    alpha = 1 - confidence_level
    tpr_ci_lower = np.percentile(tprs, 100 * alpha / 2, axis=0)
    tpr_ci_upper = np.percentile(tprs, 100 * (1 - alpha / 2), axis=0)
    
    return mean_fpr, mean_tpr, tpr_ci_lower, tpr_ci_upper


# --- 1. 计算MLP分类器的AUC置信区间 ---
print("\n1. Calculating AUC confidence interval for MLP Classifier...")
auc_mlp_mean, auc_mlp_ci_lower, auc_mlp_ci_upper, auc_mlp_std, bootstrap_aucs_mlp = \
    bootstrap_auc(y_test, y_scores_mlp, n_bootstrap=1000)

print(f"\nMLP Classifier AUC Statistics:")
print(f"  Original AUC: {roc_auc_mlp:.4f}")
print(f"  Bootstrap Mean AUC: {auc_mlp_mean:.4f} ± {auc_mlp_std:.4f}")
print(f"  95% Confidence Interval: [{auc_mlp_ci_lower:.4f}, {auc_mlp_ci_upper:.4f}]")
print(f"  CI Width: {auc_mlp_ci_upper - auc_mlp_ci_lower:.4f}")

# --- 2. 计算截断均值法的AUC置信区间 ---
print("\n2. Calculating AUC confidence interval for Truncated Mean method...")
auc_tm_mean, auc_tm_ci_lower, auc_tm_ci_upper, auc_tm_std, bootstrap_aucs_tm = \
    bootstrap_auc(y_true_truncated, y_scores_truncated, n_bootstrap=1000)

print(f"\nTruncated Mean AUC Statistics:")
print(f"  Original AUC: {roc_auc_truncated:.4f}")
print(f"  Bootstrap Mean AUC: {auc_tm_mean:.4f} ± {auc_tm_std:.4f}")
print(f"  95% Confidence Interval: [{auc_tm_ci_lower:.4f}, {auc_tm_ci_upper:.4f}]")
print(f"  CI Width: {auc_tm_ci_upper - auc_tm_ci_lower:.4f}")

# --- 3. 统计显著性检验 ---
print("\n3. Statistical Significance Test:")
print(f"  AUC Improvement: {auc_mlp_mean - auc_tm_mean:.4f}")
print(f"  Relative Improvement: {(auc_mlp_mean - auc_tm_mean) / auc_tm_mean * 100:.2f}%")

# 检查置信区间是否重叠
ci_overlap = not (auc_mlp_ci_lower > auc_tm_ci_upper or auc_tm_ci_lower > auc_mlp_ci_upper)
print(f"  95% CI Overlap: {'Yes' if ci_overlap else 'No'}")

if not ci_overlap:
    print("  ✓ The improvement is statistically significant at 95% confidence level!")
else:
    # 使用配对t检验（因为是同一批数据）
    # 注意：这里使用的是bootstrap分布进行t检验
    t_statistic, p_value = stats.ttest_ind(bootstrap_aucs_mlp, bootstrap_aucs_tm)
    print(f"  Independent t-test p-value: {p_value:.6f}")
    if p_value < 0.05:
        print("  ✓ The improvement is statistically significant (p < 0.05)!")
    else:
        print("  ✗ The improvement is NOT statistically significant (p >= 0.05)")

# --- 4. 可视化AUC分布 ---
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(bootstrap_aucs_mlp, bins=50, alpha=0.6, color='darkorange', label='MLP', density=True)
plt.hist(bootstrap_aucs_tm, bins=50, alpha=0.6, color='blue', label='Truncated Mean', density=True)
plt.axvline(auc_mlp_mean, color='darkorange', linestyle='--', linewidth=2, label=f'MLP Mean: {auc_mlp_mean:.4f}')
plt.axvline(auc_tm_mean, color='blue', linestyle='--', linewidth=2, label=f'TM Mean: {auc_tm_mean:.4f}')
plt.xlabel('AUC')
plt.ylabel('Probability Density')
plt.title('Bootstrap Distribution of AUC Values')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
methods = ['MLP Classifier', 'Truncated Mean']
means = [auc_mlp_mean, auc_tm_mean]
ci_lower = [auc_mlp_ci_lower, auc_tm_ci_lower]
ci_upper = [auc_mlp_ci_upper, auc_tm_ci_upper]
errors = [[auc_mlp_mean - auc_mlp_ci_lower], [auc_mlp_ci_upper - auc_mlp_mean]]

x_pos = np.arange(len(methods))
plt.bar(x_pos, means, yerr=[[means[i] - ci_lower[i] for i in range(2)], 
                              [ci_upper[i] - means[i] for i in range(2)]], 
        align='center', alpha=0.7, ecolor='black', capsize=10,
        color=['darkorange', 'blue'])
plt.ylabel('AUC')
plt.xticks(x_pos, methods)
plt.title('AUC Comparison with 95% Confidence Intervals')
plt.ylim([0.75, 0.85])
plt.grid(True, alpha=0.3, axis='y')

# 添加数值标签
for i, (method, mean, lower, upper) in enumerate(zip(methods, means, ci_lower, ci_upper)):
    plt.text(i, upper + 0.005, f'{mean:.4f}\n±{(upper-lower)/2:.4f}', 
             ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.savefig('auc_statistical_significance_analysis.png', dpi=300)
plt.show()

# --- 5. 计算并绘制带置信区间的ROC曲线 ---
print("\n4. Computing ROC curves with confidence bands...")

# 为MLP计算ROC置信带
mean_fpr_mlp, mean_tpr_mlp, tpr_ci_lower_mlp, tpr_ci_upper_mlp = \
    bootstrap_roc_curve(y_test, y_scores_mlp, n_bootstrap=1000)

# 为截断均值法计算ROC置信带
mean_fpr_tm, mean_tpr_tm, tpr_ci_lower_tm, tpr_ci_upper_tm = \
    bootstrap_roc_curve(y_true_truncated, y_scores_truncated, n_bootstrap=1000)

# --- 6. 绘制增强版ROC曲线（带置信区间） ---
plt.figure(figsize=(12, 10))

# 原始ROC曲线作为对比
plt.plot(fpr_mlp, tpr_mlp, color='darkorange', lw=1, alpha=0.3, 
         label=f'MLP Original (AUC = {roc_auc_mlp:.4f})')
plt.plot(fpr_truncated, tpr_truncated, color='blue', lw=1, alpha=0.3,
         label=f'TM Original (AUC = {roc_auc_truncated:.4f})')

# 平均ROC曲线（粗线）
plt.plot(mean_fpr_mlp, mean_tpr_mlp, color='darkorange', lw=3,
         label=f'MLP Mean (AUC = {auc_mlp_mean:.4f} [{auc_mlp_ci_lower:.4f}, {auc_mlp_ci_upper:.4f}])')
plt.plot(mean_fpr_tm, mean_tpr_tm, color='blue', lw=3, linestyle='--',
         label=f'TM Mean (AUC = {auc_tm_mean:.4f} [{auc_tm_ci_lower:.4f}, {auc_tm_ci_upper:.4f}])')

# 95%置信区间（阴影带）
plt.fill_between(mean_fpr_mlp, tpr_ci_lower_mlp, tpr_ci_upper_mlp, 
                 color='darkorange', alpha=0.2, label='MLP 95% CI')
plt.fill_between(mean_fpr_tm, tpr_ci_lower_tm, tpr_ci_upper_tm, 
                 color='blue', alpha=0.2, label='TM 95% CI')

# 对角线（随机猜测）
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle=':', label='Random Guess')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (Proton identified as Pion)', fontsize=12)
plt.ylabel('True Positive Rate (Proton identified as Proton)', fontsize=12)
plt.title(f'ROC Curves with 95% Confidence Intervals\n(Pion vs Proton, p={MOMENTUM_OVERLAP_MIN}-{MOMENTUM_OVERLAP_MAX} GeV/c)', 
          fontsize=14)
plt.legend(loc="lower right", fontsize=10)
plt.grid(True, alpha=0.3)

# 添加文本框显示统计信息
textstr = f'Statistical Significance:\n'
textstr += f'ΔAUC = {auc_mlp_mean - auc_tm_mean:.4f}\n'
textstr += f'Relative Improvement = {(auc_mlp_mean - auc_tm_mean) / auc_tm_mean * 100:.2f}%\n'
if not ci_overlap:
    textstr += f'95% CIs do NOT overlap ✓'
else:
    textstr += f'p-value = {p_value:.4f}'
    
props = dict(boxstyle='round', facecolor='wheat', alpha=0.8)
plt.text(0.60, 0.15, textstr, transform=plt.gca().transAxes, fontsize=11,
         verticalalignment='top', bbox=props)

plt.tight_layout()
plt.savefig('roc_curves_with_confidence_intervals.png', dpi=300)
plt.show()

print("\n=== Analysis Complete ===")
print("Generated files:")
print("  1. auc_statistical_significance_analysis.png")
print("  2. roc_curves_with_confidence_intervals.png")

# --- 7. 生成详细的统计报告 ---
print("\n" + "="*60)
print("FINAL STATISTICAL REPORT")
print("="*60)
print(f"\nMethod Comparison (p = {MOMENTUM_OVERLAP_MIN}-{MOMENTUM_OVERLAP_MAX} GeV/c):")
print("-" * 60)
print(f"{'Method':<20} {'AUC':<12} {'95% CI':<25} {'Std Dev':<10}")
print("-" * 60)
print(f"{'MLP Classifier':<20} {auc_mlp_mean:.4f}      [{auc_mlp_ci_lower:.4f}, {auc_mlp_ci_upper:.4f}]    {auc_mlp_std:.4f}")
print(f"{'Truncated Mean':<20} {auc_tm_mean:.4f}      [{auc_tm_ci_lower:.4f}, {auc_tm_ci_upper:.4f}]    {auc_tm_std:.4f}")
print("-" * 60)
print(f"\nImprovement: {auc_mlp_mean - auc_tm_mean:.4f} ({(auc_mlp_mean - auc_tm_mean) / auc_tm_mean * 100:.2f}%)")
print(f"Significance: {'YES (CIs do not overlap)' if not ci_overlap else f'p-value = {p_value:.4f}'}")
print("="*60)

# （二）特征理解
2.1.6特征重要性分析


2.2.4输入特征分布可视化

In [ ]:
print("\n" + "="*80)
print("SUPPLEMENTARY ANALYSIS: Feature Importance and Distribution")
print("="*80)

import numpy as np
import matplotlib.pyplot as plt
from sklearn.inspection import permutation_importance
from sklearn.base import BaseEstimator, ClassifierMixin
import seaborn as sns
from scipy import stats as scipy_stats

# --- 第一部分：特征重要性分析（Permutation Importance） ---
print("\n[1/2] Computing Feature Importance using Permutation Importance...")
print("This method measures how much the model performance drops when a feature is randomly shuffled.")

# 创建一个符合sklearn规范的包装器
class PyTorchClassifierWrapper(BaseEstimator, ClassifierMixin):
    """包装PyTorch模型使其完全兼容sklearn"""
    def __init__(self, model, device, scaler):
        self.model = model
        self.device = device
        self.scaler = scaler
        self.classes_ = np.array([0, 1])  # 二分类
        self.model.eval()
    
    def fit(self, X, y):
        """占位符fit方法（模型已经训练好了）"""
        return self
    
    def predict_proba(self, X):
        """预测概率"""
        with torch.no_grad():
            X_scaled = self.scaler.transform(X)
            X_tensor = torch.tensor(X_scaled, dtype=torch.float32).to(self.device)
            outputs = self.model(X_tensor).cpu().numpy().flatten()
        # 返回[P(class=0), P(class=1)]
        proba = np.column_stack([1 - outputs, outputs])
        return proba
    
    def predict(self, X):
        """预测类别"""
        proba = self.predict_proba(X)
        return (proba[:, 1] > 0.5).astype(int)
    
    def score(self, X, y):
        """计算准确率"""
        from sklearn.metrics import accuracy_score
        return accuracy_score(y, self.predict(X))

# 创建包装器
wrapped_model = PyTorchClassifierWrapper(model_classifier, device, scaler_classifier)

# 计算排列重要性（使用测试集）
print("  Running permutation importance (this may take a few minutes)...")
perm_importance = permutation_importance(
    wrapped_model, 
    X_test,  # 使用未缩放的测试数据
    y_test, 
    n_repeats=30,  # 重复30次以获得稳定结果
    random_state=42,
    scoring='roc_auc',  # 使用AUC作为评分标准
    n_jobs=-1  # 使用所有CPU核心
)

# 提取特征名称
# 前50个是排序后的dE/dx值，后5个是统计特征
feature_names = [f'dE/dx_{i+1}' for i in range(NUM_DE_DX_VALUES)] + \
                ['Mean', 'Std Dev', 'Median', 'Skewness', 'Kurtosis']

# 创建特征重要性DataFrame
importance_df = {
    'Feature': feature_names,
    'Importance': perm_importance.importances_mean,
    'Std': perm_importance.importances_std
}

# 按重要性排序
sorted_idx = np.argsort(perm_importance.importances_mean)[::-1]

print("\n  Top 15 Most Important Features:")
print("  " + "-"*70)
print(f"  {'Rank':<6} {'Feature':<20} {'Importance':<15} {'Std Dev':<10}")
print("  " + "-"*70)
for i, idx in enumerate(sorted_idx[:15], 1):
    print(f"  {i:<6} {feature_names[idx]:<20} {perm_importance.importances_mean[idx]:.6f}     "
          f"±{perm_importance.importances_std[idx]:.6f}")

# 分析统计特征的重要性
stat_features_idx = list(range(NUM_DE_DX_VALUES, NUM_DE_DX_VALUES + 5))
stat_features_importance = perm_importance.importances_mean[stat_features_idx]
stat_features_names = ['Mean', 'Std Dev', 'Median', 'Skewness', 'Kurtosis']

print("\n  Statistical Features Importance Summary:")
print("  " + "-"*70)
for name, imp, std in zip(stat_features_names, stat_features_importance, 
                           perm_importance.importances_std[stat_features_idx]):
    print(f"  {name:<15} Importance: {imp:.6f} ± {std:.6f}")
print("  " + "-"*70)

# ====== 图1：Top 20 特征重要性（单图） ======
fig, ax = plt.subplots(1, 1, figsize=(10, 8))

top_n = 20
top_indices = sorted_idx[:top_n]
y_pos = np.arange(top_n)
colors = ['red' if idx >= NUM_DE_DX_VALUES else 'steelblue' for idx in top_indices]

ax.barh(y_pos, perm_importance.importances_mean[top_indices], 
        xerr=perm_importance.importances_std[top_indices],
        color=colors, alpha=0.7, capsize=3)
ax.set_yticks(y_pos)
ax.set_yticklabels([feature_names[i] for i in top_indices])
ax.invert_yaxis()
ax.set_xlabel('Permutation Importance (ΔAUC)', fontsize=12)
ax.set_title(f'Top {top_n} Most Important Features for Pion/Proton Classification', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')

# 创建图例
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor='red', alpha=0.7, label='Statistical Features'),
                   Patch(facecolor='steelblue', alpha=0.7, label='dE/dx Values')]
ax.legend(handles=legend_elements, loc='lower right', fontsize=10)

plt.tight_layout()
plt.savefig('feature_importance_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n  ✓ Feature importance plot saved as 'feature_importance_analysis.png'")

# --- 第二部分：特征分布可视化 ---
print("\n[2/2] Visualizing Statistical Feature Distributions for Pions vs Protons...")

# 从测试集中提取π介子和质子的数据
pion_mask_test = y_test == 0
proton_mask_test = y_test == 1

# 提取统计特征（最后5列）
stat_features_test = X_test[:, -5:]  # 未缩放的测试数据的最后5列
stat_features_pions = stat_features_test[pion_mask_test]
stat_features_protons = stat_features_test[proton_mask_test]

print(f"  Number of pions in test set: {np.sum(pion_mask_test)}")
print(f"  Number of protons in test set: {np.sum(proton_mask_test)}")

# ====== 图2：5个统计特征的分布对比 ======
fig, axes = plt.subplots(3, 2, figsize=(14, 14))
axes = axes.flatten()

stat_feature_labels = ['Mean dE/dx', 'Std Dev of dE/dx', 'Median dE/dx', 
                       'Skewness of dE/dx', 'Kurtosis of dE/dx']

for i, (label, importance) in enumerate(zip(stat_feature_labels, stat_features_importance)):
    ax = axes[i]
    
    # 绘制直方图
    ax.hist(stat_features_pions[:, i], bins=50, alpha=0.5, label='Pion (π)', 
            color='blue', density=True)
    ax.hist(stat_features_protons[:, i], bins=50, alpha=0.5, label='Proton (p)', 
            color='red', density=True)
    
    # 添加均值线
    pion_mean = np.mean(stat_features_pions[:, i])
    proton_mean = np.mean(stat_features_protons[:, i])
    ax.axvline(pion_mean, color='blue', linestyle='--', linewidth=2, alpha=0.7)
    ax.axvline(proton_mean, color='red', linestyle='--', linewidth=2, alpha=0.7)
    
    # 计算分离度（效应量：Cohen's d）
    pooled_std = np.sqrt((np.std(stat_features_pions[:, i])**2 + 
                          np.std(stat_features_protons[:, i])**2) / 2)
    cohens_d = (proton_mean - pion_mean) / pooled_std
    
    # 进行Kolmogorov-Smirnov检验
    ks_statistic, ks_pvalue = scipy_stats.ks_2samp(stat_features_pions[:, i], 
                                                     stat_features_protons[:, i])
    
    ax.set_xlabel(label, fontsize=11)
    ax.set_ylabel('Probability Density', fontsize=11)
    ax.set_title(f'{label}\nImportance: {importance:.5f} | Cohen\'s d: {cohens_d:.3f}', 
                 fontsize=10)
    ax.legend(loc='best', fontsize=9)
    ax.grid(True, alpha=0.3)
    
    # 添加文本框显示统计信息
    textstr = f'Pion μ={pion_mean:.3f}\nProton μ={proton_mean:.3f}\n'
    textstr += f'KS test p<{ks_pvalue:.1e}'
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.7)
    ax.text(0.98, 0.97, textstr, transform=ax.transAxes, fontsize=8,
            verticalalignment='top', horizontalalignment='right', bbox=props)

# 删除第6个子图（因为只有5个特征）
fig.delaxes(axes[5])

plt.suptitle(f'Statistical Feature Distributions: Pions vs Protons\n'
             f'(Momentum range: {MOMENTUM_OVERLAP_MIN}-{MOMENTUM_OVERLAP_MAX} GeV/c)', 
             fontsize=14, y=0.995)
plt.tight_layout(rect=[0, 0, 1, 0.99])
plt.savefig('statistical_features_distribution_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("  ✓ Feature distribution plot saved as 'statistical_features_distribution_comparison.png'")

# 计算每个统计特征的Cohen's d
cohens_d_values = []
for i in range(5):
    pion_mean = np.mean(stat_features_pions[:, i])
    proton_mean = np.mean(stat_features_protons[:, i])
    pooled_std = np.sqrt((np.std(stat_features_pions[:, i])**2 + 
                          np.std(stat_features_protons[:, i])**2) / 2)
    cohens_d = (proton_mean - pion_mean) / pooled_std
    cohens_d_values.append(cohens_d)

print("a) Mean dE/dx:")
print(f"   - Importance: {stat_features_importance[0]:.6f}")
print(f"   - Cohen's d: {cohens_d_values[0]:.3f}")
print("   - Physics: Central tendency of energy loss, directly related to Bethe-Bloch")
print()
print("b) Standard Deviation:")
print(f"   - Importance: {stat_features_importance[1]:.6f}")
print(f"   - Cohen's d: {cohens_d_values[1]:.3f}")
print("   - Physics: Captures fluctuations in energy loss (Landau distribution)")
print()
print("c) Median dE/dx:")
print(f"   - Importance: {stat_features_importance[2]:.6f}")
print(f"   - Cohen's d: {cohens_d_values[2]:.3f}")
print("   - Physics: Robust central measure, less affected by outliers")
print()
print("d) Skewness:")
print(f"   - Importance: {stat_features_importance[3]:.6f}")
print(f"   - Cohen's d: {cohens_d_values[3]:.3f}")
print("   - Physics: Asymmetry of energy loss distribution (Landau tail)")
print()

# （三）模型评估扩展
2.1.4扩展到其他粒子对和动量范围

In [ ]:
print("\n" + "="*80)
print("EXTENDED ANALYSIS: Multi-Particle-Pair & Multi-Momentum-Range Evaluation")
print("="*80)

import itertools
import pandas as pd
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
import warnings
warnings.filterwarnings('ignore')

# --- 1. 定义要测试的粒子对和动量范围 ---
# 根据数据集中的质量值定义粒子
PARTICLE_DEFINITIONS = {
    'Electron': 0.001,
    'Pion': 0.139,
    'Kaon': 0.494,
    'Proton': 0.937
}

# 定义要测试的粒子对（选择物理上重要的组合）
PARTICLE_PAIRS = [
    ('Pion', 'Proton'),      # 原始测试
    ('Kaon', 'Proton'),      # 高动量区域重要
    ('Pion', 'Kaon'),        # 中等动量区域挑战
    ('Electron', 'Pion'),    # 低动量区域
]

# 定义多个动量范围（覆盖不同物理区域）
MOMENTUM_RANGES = [
    (0.3, 0.6),   # 低动量（粒子明显分离）
    (0.6, 1.0),   # 中低动量
    (1.0, 1.5),   # 中等动量（原始范围的前半段）
    (1.5, 2.0),   # 中高动量（原始范围的后半段）
    (2.0, 2.5),   # 高动量（分离困难）
]

print(f"\nTesting Configuration:")
print(f"  Particle pairs: {len(PARTICLE_PAIRS)}")
print(f"  Momentum ranges: {len(MOMENTUM_RANGES)}")
print(f"  Total test cases: {len(PARTICLE_PAIRS) * len(MOMENTUM_RANGES)}")

# --- 2. 准备数据提取函数 ---
def extract_particle_pair_data(mass_data, momentum_data, X_data, y_truncated_data,
                                particle1_name, particle2_name, p_min, p_max,
                                particle_defs=PARTICLE_DEFINITIONS):
    """
    从全量数据中提取特定粒子对在特定动量范围的数据
    
    返回: X_pair, y_pair, p_pair, dedx_pair, n_particle1, n_particle2
    """
    mass1 = particle_defs[particle1_name]
    mass2 = particle_defs[particle2_name]
    
    # 筛选粒子1
    mask1 = (mass_data == mass1) & (momentum_data >= p_min) & (momentum_data < p_max)
    X1 = X_data[mask1]
    p1 = momentum_data[mask1]
    dedx1 = y_truncated_data[mask1]
    
    # 筛选粒子2
    mask2 = (mass_data == mass2) & (momentum_data >= p_min) & (momentum_data < p_max)
    X2 = X_data[mask2]
    p2 = momentum_data[mask2]
    dedx2 = y_truncated_data[mask2]
    
    # 检查数据量
    n1, n2 = len(X1), len(X2)
    if n1 < 100 or n2 < 100:
        return None, None, None, None, n1, n2
    
    # 合并数据 (粒子1标签为0，粒子2标签为1)
    X_pair = np.vstack((X1, X2))
    y_pair = np.hstack((np.zeros(n1), np.ones(n2)))
    p_pair = np.hstack((p1, p2))
    dedx_pair = np.hstack((dedx1, dedx2))
    
    return X_pair, y_pair, p_pair, dedx_pair, n1, n2


# --- 3. 为每个测试用例训练和评估模型 ---
results_comprehensive = []

print("\nStarting comprehensive evaluation...")
print("-" * 80)

test_case_count = 0
for (particle1, particle2), (p_min, p_max) in itertools.product(PARTICLE_PAIRS, MOMENTUM_RANGES):
    test_case_count += 1
    print(f"\n[{test_case_count}/{len(PARTICLE_PAIRS) * len(MOMENTUM_RANGES)}] "
          f"Testing: {particle1} vs {particle2}, p ∈ [{p_min}, {p_max}) GeV/c")
    
    # 提取数据
    X_pair, y_pair, p_pair, dedx_pair, n1, n2 = extract_particle_pair_data(
        mass_data_full, momentum_data_full, X_data_full, y_truncated_mean_full,
        particle1, particle2, p_min, p_max
    )
    
    if X_pair is None:
        print(f"  ✗ Insufficient data: {particle1}={n1}, {particle2}={n2} (need ≥100 each)")
        results_comprehensive.append({
            'Particle_Pair': f'{particle1}-{particle2}',
            'Momentum_Range': f'{p_min}-{p_max}',
            'N_Particle1': n1,
            'N_Particle2': n2,
            'Status': 'Insufficient Data',
            'MLP_AUC': np.nan,
            'TruncMean_AUC': np.nan,
            'AUC_Improvement': np.nan,
            'MLP_Accuracy': np.nan,
            'TruncMean_Accuracy': np.nan
        })
        continue
    
    print(f"  Data: {particle1}={n1}, {particle2}={n2}")
    
    # 数据标准化和划分
    scaler_temp = StandardScaler()
    X_pair_scaled = scaler_temp.fit_transform(X_pair)
    
    X_train_temp, X_test_temp, y_train_temp, y_test_temp, \
    p_train_temp, p_test_temp, dedx_train_temp, dedx_test_temp = train_test_split(
        X_pair_scaled, y_pair, p_pair, dedx_pair,
        test_size=0.2, random_state=42, stratify=y_pair
    )
    
    # 转换为Tensor
    X_train_tensor_temp = torch.tensor(X_train_temp, dtype=torch.float32).to(device)
    y_train_tensor_temp = torch.tensor(y_train_temp, dtype=torch.float32).to(device).reshape(-1, 1)
    X_test_tensor_temp = torch.tensor(X_test_temp, dtype=torch.float32).to(device)
    
    # 创建DataLoader
    train_dataset_temp = TensorDataset(X_train_tensor_temp, y_train_tensor_temp)
    train_loader_temp = DataLoader(train_dataset_temp, batch_size=256, shuffle=True)
    
    # 训练新的MLP模型（使用相同架构）
    model_temp = MLPClassifierOptimized(input_size, hidden_size_classifier).to(device)
    criterion_temp = nn.BCELoss()
    optimizer_temp = optim.Adam(model_temp.parameters(), lr=0.001, weight_decay=1e-5)
    
    # 快速训练（50个epoch，无早停以节省时间）
    print("  Training MLP...", end=" ")
    for epoch in range(50):
        model_temp.train()
        for inputs, labels in train_loader_temp:
            optimizer_temp.zero_grad()
            outputs = model_temp(inputs)
            loss = criterion_temp(outputs, labels)
            loss.backward()
            optimizer_temp.step()
    
    # MLP预测和评估
    model_temp.eval()
    with torch.no_grad():
        y_scores_mlp_temp = model_temp(X_test_tensor_temp).cpu().numpy().flatten()
    
    auc_mlp_temp = roc_auc_score(y_test_temp, y_scores_mlp_temp)
    y_pred_mlp_temp = (y_scores_mlp_temp >= 0.5).astype(int)
    acc_mlp_temp = accuracy_score(y_test_temp, y_pred_mlp_temp)
    
    # 截断均值法评估（使用dedx作为分数）
    auc_truncmean_temp = roc_auc_score(y_test_temp, dedx_test_temp)
    # 使用中位数作为阈值进行二分类
    threshold_truncmean = np.median(dedx_test_temp)
    y_pred_truncmean_temp = (dedx_test_temp >= threshold_truncmean).astype(int)
    acc_truncmean_temp = accuracy_score(y_test_temp, y_pred_truncmean_temp)
    
    improvement = auc_mlp_temp - auc_truncmean_temp
    
    print(f"Done! MLP AUC={auc_mlp_temp:.4f}, TruncMean AUC={auc_truncmean_temp:.4f}, "
          f"Δ={improvement:.4f} ({improvement/auc_truncmean_temp*100:+.2f}%)")
    
    # 保存结果
    results_comprehensive.append({
        'Particle_Pair': f'{particle1}-{particle2}',
        'Momentum_Range': f'{p_min}-{p_max}',
        'N_Particle1': n1,
        'N_Particle2': n2,
        'Status': 'Success',
        'MLP_AUC': auc_mlp_temp,
        'TruncMean_AUC': auc_truncmean_temp,
        'AUC_Improvement': improvement,
        'Relative_Improvement_%': improvement / auc_truncmean_temp * 100,
        'MLP_Accuracy': acc_mlp_temp,
        'TruncMean_Accuracy': acc_truncmean_temp
    })

print("\n" + "="*80)
print("COMPREHENSIVE EVALUATION COMPLETED")
print("="*80)

# --- 4. 创建结果DataFrame并分析 ---
df_results = pd.DataFrame(results_comprehensive)

# 分离成功和失败的测试用例
df_success = df_results[df_results['Status'] == 'Success'].copy()
df_failed = df_results[df_results['Status'] != 'Success'].copy()

print(f"\nTest Summary:")
print(f"  Successful tests: {len(df_success)}/{len(df_results)}")
print(f"  Failed tests (insufficient data): {len(df_failed)}")

if len(df_success) > 0:
    print(f"\nOverall Statistics (successful tests only):")
    print(f"  Average MLP AUC: {df_success['MLP_AUC'].mean():.4f} ± {df_success['MLP_AUC'].std():.4f}")
    print(f"  Average TruncMean AUC: {df_success['TruncMean_AUC'].mean():.4f} ± {df_success['TruncMean_AUC'].std():.4f}")
    print(f"  Average Improvement: {df_success['AUC_Improvement'].mean():.4f} ({df_success['Relative_Improvement_%'].mean():.2f}%)")
    print(f"  MLP wins in: {(df_success['AUC_Improvement'] > 0).sum()}/{len(df_success)} cases")
    
    # 保存详细结果到CSV
    df_results.to_csv('comprehensive_evaluation_results.csv', index=False)
    print(f"\n✓ Detailed results saved to 'comprehensive_evaluation_results.csv'")

# --- 5. 可视化：热图矩阵（最核心的图） ---
print("\nGenerating performance heatmap...")

# 为热图准备数据（只包含成功的测试）
if len(df_success) > 0:
    # 创建数据透视表：粒子对 vs 动量范围
    heatmap_data_improvement = df_success.pivot_table(
        values='AUC_Improvement', 
        index='Particle_Pair', 
        columns='Momentum_Range',
        aggfunc='mean'
    )
    
    heatmap_data_mlp_auc = df_success.pivot_table(
        values='MLP_AUC',
        index='Particle_Pair',
        columns='Momentum_Range',
        aggfunc='mean'
    )
    
    # 绘制热图
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 6))
    
    # 子图1：AUC改进热图（最关键）
    sns.heatmap(heatmap_data_improvement, annot=True, fmt='.4f', cmap='RdYlGn',
                center=0, vmin=-0.05, vmax=0.05, cbar_kws={'label': 'ΔAUC (MLP - TruncMean)'},
                linewidths=0.5, ax=ax1)
    ax1.set_title('MLP Performance Improvement Over Truncated Mean\n(Green=Better, Red=Worse)', 
                  fontsize=13, fontweight='bold')
    ax1.set_xlabel('Momentum Range (GeV/c)', fontsize=11)
    ax1.set_ylabel('Particle Pair', fontsize=11)
    
    # 子图2：MLP绝对AUC值热图
    sns.heatmap(heatmap_data_mlp_auc, annot=True, fmt='.3f', cmap='YlOrRd',
                vmin=0.5, vmax=1.0, cbar_kws={'label': 'MLP AUC'},
                linewidths=0.5, ax=ax2)
    ax2.set_title('MLP Absolute Performance (AUC)\n(Darker=Better Classification)', 
                  fontsize=13, fontweight='bold')
    ax2.set_xlabel('Momentum Range (GeV/c)', fontsize=11)
    ax2.set_ylabel('Particle Pair', fontsize=11)
    
    plt.tight_layout()
    plt.savefig('comprehensive_performance_heatmap.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✓ Performance heatmap saved as 'comprehensive_performance_heatmap.png'")

# --- 6. 可视化：条形图对比（按粒子对分组） ---
if len(df_success) > 0:
    print("\nGenerating grouped bar chart...")
    
    fig, ax = plt.subplots(figsize=(14, 7))
    
    # 按粒子对分组
    particle_pairs_unique = df_success['Particle_Pair'].unique()
    x = np.arange(len(particle_pairs_unique))
    width = 0.35
    
    # 计算每个粒子对的平均AUC
    mlp_avg = [df_success[df_success['Particle_Pair']==pair]['MLP_AUC'].mean() 
               for pair in particle_pairs_unique]
    truncmean_avg = [df_success[df_success['Particle_Pair']==pair]['TruncMean_AUC'].mean() 
                     for pair in particle_pairs_unique]
    
    bars1 = ax.bar(x - width/2, mlp_avg, width, label='MLP Classifier', 
                   color='darkorange', alpha=0.8)
    bars2 = ax.bar(x + width/2, truncmean_avg, width, label='Truncated Mean', 
                   color='steelblue', alpha=0.8)
    
    # 添加数值标签
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:.3f}', ha='center', va='bottom', fontsize=9)
    
    ax.set_xlabel('Particle Pair', fontsize=12)
    ax.set_ylabel('Average AUC (across all momentum ranges)', fontsize=12)
    ax.set_title('MLP vs Truncated Mean: Performance Comparison by Particle Pair', 
                 fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(particle_pairs_unique, fontsize=10)
    ax.legend(fontsize=11)
    ax.grid(True, alpha=0.3, axis='y')
    ax.set_ylim(0.5, 1.0)
    
    plt.tight_layout()
    plt.savefig('comprehensive_performance_by_particle_pair.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✓ Grouped bar chart saved as 'comprehensive_performance_by_particle_pair.png'")

# --- 7. 生成最终报告 ---
print("\n" + "="*80)
print("FINAL COMPREHENSIVE REPORT")
print("="*80)

if len(df_success) > 0:
    print("\nKey Findings:")
    print("-" * 80)
    
    # 找出MLP表现最好和最差的场景
    best_case = df_success.loc[df_success['AUC_Improvement'].idxmax()]
    worst_case = df_success.loc[df_success['AUC_Improvement'].idxmin()]
    
    print(f"\n1. Best Performance Improvement:")
    print(f"   Particle Pair: {best_case['Particle_Pair']}")
    print(f"   Momentum Range: {best_case['Momentum_Range']} GeV/c")
    print(f"   Improvement: ΔAUC = {best_case['AUC_Improvement']:.4f} ({best_case['Relative_Improvement_%']:.2f}%)")
    print(f"   MLP AUC: {best_case['MLP_AUC']:.4f} vs TruncMean: {best_case['TruncMean_AUC']:.4f}")
    
    print(f"\n2. Worst Performance (or smallest improvement):")
    print(f"   Particle Pair: {worst_case['Particle_Pair']}")
    print(f"   Momentum Range: {worst_case['Momentum_Range']} GeV/c")
    print(f"   Improvement: ΔAUC = {worst_case['AUC_Improvement']:.4f} ({worst_case['Relative_Improvement_%']:.2f}%)")
    print(f"   MLP AUC: {worst_case['MLP_AUC']:.4f} vs TruncMean: {worst_case['TruncMean_AUC']:.4f}")
    
    print(f"\n3. Consistency Analysis:")
    positive_improvements = (df_success['AUC_Improvement'] > 0).sum()
    print(f"   MLP outperforms TruncMean in {positive_improvements}/{len(df_success)} scenarios")
    
    significant_improvements = (df_success['AUC_Improvement'] > 0.01).sum()
    print(f"   Significant improvements (ΔAUC > 0.01): {significant_improvements}/{len(df_success)} scenarios")
    
    # 按动量范围分析
    print(f"\n4. Performance by Momentum Range:")
    for mom_range in df_success['Momentum_Range'].unique():
        subset = df_success[df_success['Momentum_Range'] == mom_range]
        avg_improvement = subset['AUC_Improvement'].mean()
        print(f"   {mom_range} GeV/c: Average ΔAUC = {avg_improvement:.4f} ({len(subset)} pairs tested)")
    
    print("\n" + "="*80)
    print("CONCLUSION:")
    print("="*80)
    
    overall_improvement = df_success['AUC_Improvement'].mean()
    if overall_improvement > 0.01:
        print(f"✓ The MLP classifier demonstrates CONSISTENT and SIGNIFICANT superiority")
        print(f"  over the traditional truncated mean method across diverse scenarios.")
        print(f"  Average improvement: ΔAUC = {overall_improvement:.4f} ({df_success['Relative_Improvement_%'].mean():.2f}%)")
    elif overall_improvement > 0:
        print(f"✓ The MLP classifier shows MODEST bukt CONSISTENT improvement")
        print(f"  over the traditional method in most scenarios.")
        print(f"  Average improvement: ΔAUC = {overall_improvement:.4f}")
    else:
        print(f"⚠ The MLP classifier does NOT consistently outperform the traditional method.")
        print(f"  Further optimization or feature engineering may be needed.")
    
    print("="*80)

print("\n✓ Comprehensive evaluation complete!")
print("  Generated files:")
print("  - comprehensive_evaluation_results.csv")
print("  - comprehensive_performance_heatmap.png")
print("  - comprehensive_performance_by_particle_pair.png")

# 2.1.5 与其他机器学习模型的对比

In [ ]:
print("\n" + "="*80)
print("BENCHMARK ANALYSIS: MLP vs XGBoost vs Random Forest")
print("="*80)

import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, roc_curve, auc
import time
import pandas as pd
from scipy import stats

# --- 1. 使用原始数据（Pion vs Proton, 1.0-2.0 GeV/c）---
print("\n[Step 1/5] Using existing data from previous analysis...")
print(f"  Training samples: {len(X_train)}")
print(f"  Test samples: {len(X_test)}")
print(f"  Features: {X_train.shape[1]}")

# --- 2. 训练XGBoost模型 ---
print("\n[Step 2/5] Training XGBoost Classifier...")
start_time = time.time()

xgb_model = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='binary:logistic',
    random_state=42,
    eval_metric='auc',
    early_stopping_rounds=15,
    tree_method='hist',  # 使用直方图优化加速
    device='cuda' if torch.cuda.is_available() else 'cpu'  # GPU加速
)

# 训练XGBoost（使用验证集进行早停）
xgb_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=False
)

xgb_train_time = time.time() - start_time
print(f"  ✓ XGBoost training completed in {xgb_train_time:.2f}s")
print(f"  Best iteration: {xgb_model.best_iteration}")

# XGBoost预测
y_scores_xgb = xgb_model.predict_proba(X_test)[:, 1]
y_pred_xgb = (y_scores_xgb >= 0.5).astype(int)
auc_xgb = roc_auc_score(y_test, y_scores_xgb)
acc_xgb = accuracy_score(y_test, y_pred_xgb)

print(f"  XGBoost Test AUC: {auc_xgb:.4f}")
print(f"  XGBoost Test Accuracy: {acc_xgb:.4f}")

# --- 3. 训练Random Forest模型 ---
print("\n[Step 3/5] Training Random Forest Classifier...")
start_time = time.time()

rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=15,
    min_samples_split=10,
    min_samples_leaf=5,
    max_features='sqrt',
    bootstrap=True,
    random_state=42,
    n_jobs=-1,  # 使用所有CPU核心并行训练
    verbose=0
)

rf_model.fit(X_train, y_train)
rf_train_time = time.time() - start_time
print(f"  ✓ Random Forest training completed in {rf_train_time:.2f}s")
print(f"  Number of trees: {rf_model.n_estimators}")

# Random Forest预测
y_scores_rf = rf_model.predict_proba(X_test)[:, 1]
y_pred_rf = (y_scores_rf >= 0.5).astype(int)
auc_rf = roc_auc_score(y_test, y_scores_rf)
acc_rf = accuracy_score(y_test, y_pred_rf)

print(f"  Random Forest Test AUC: {auc_rf:.4f}")
print(f"  Random Forest Test Accuracy: {acc_rf:.4f}")

# --- 4. 收集所有模型的性能指标 ---
print("\n[Step 4/5] Computing Bootstrap Confidence Intervals for all models...")

# 为XGBoost和Random Forest计算Bootstrap置信区间
print("  Computing XGBoost confidence intervals...")
auc_xgb_mean, auc_xgb_ci_lower, auc_xgb_ci_upper, auc_xgb_std, bootstrap_aucs_xgb = \
    bootstrap_auc(y_test, y_scores_xgb, n_bootstrap=1000)

print("  Computing Random Forest confidence intervals...")
auc_rf_mean, auc_rf_ci_lower, auc_rf_ci_upper, auc_rf_std, bootstrap_aucs_rf = \
    bootstrap_auc(y_test, y_scores_rf, n_bootstrap=1000)

# 整理所有结果
results_benchmark = {
    'Model': ['MLP', 'XGBoost', 'Random Forest', 'Truncated Mean'],
    'AUC': [auc_mlp_mean, auc_xgb_mean, auc_rf_mean, auc_tm_mean],
    'AUC_CI_Lower': [auc_mlp_ci_lower, auc_xgb_ci_lower, auc_rf_ci_lower, auc_tm_ci_lower],
    'AUC_CI_Upper': [auc_mlp_ci_upper, auc_xgb_ci_upper, auc_rf_ci_upper, auc_tm_ci_upper],
    'AUC_Std': [auc_mlp_std, auc_xgb_std, auc_rf_std, auc_tm_std],
    'Accuracy': [acc_mlp_temp, acc_xgb, acc_rf, acc_truncmean_temp],
    'Training_Time_s': [np.nan, xgb_train_time, rf_train_time, 0]  # MLP训练时间未单独记录
}

df_benchmark = pd.DataFrame(results_benchmark)

# --- 5. 统计显著性检验（成对比较）---
print("\n[Step 5/5] Performing Statistical Significance Tests...")
print("  Pairwise Comparisons (Independent t-tests on Bootstrap AUC distributions):")
print("  " + "-"*70)

models_dict = {
    'MLP': bootstrap_aucs_mlp,
    'XGBoost': bootstrap_aucs_xgb,
    'Random Forest': bootstrap_aucs_rf,
    'Truncated Mean': bootstrap_aucs_tm
}

comparisons = []
for model1, model2 in [('MLP', 'XGBoost'), ('MLP', 'Random Forest'), 
                       ('XGBoost', 'Random Forest'), ('MLP', 'Truncated Mean'),
                       ('XGBoost', 'Truncated Mean'), ('Random Forest', 'Truncated Mean')]:
    t_stat, p_value = stats.ttest_ind(models_dict[model1], models_dict[model2])
    mean_diff = np.mean(models_dict[model1]) - np.mean(models_dict[model2])
    
    # 效应量 (Cohen's d)
    pooled_std = np.sqrt((np.std(models_dict[model1])**2 + 
                          np.std(models_dict[model2])**2) / 2)
    cohens_d = mean_diff / pooled_std
    
    significance = "***" if p_value < 0.001 else ("**" if p_value < 0.01 else ("*" if p_value < 0.05 else "n.s."))
    
    print(f"  {model1:16} vs {model2:16}: ΔAUC={mean_diff:+.4f}, p={p_value:.6f} {significance}, d={cohens_d:+.3f}")
    
    comparisons.append({
        'Comparison': f'{model1} vs {model2}',
        'Mean_Difference': mean_diff,
        'p_value': p_value,
        'Cohens_d': cohens_d,
        'Significant': p_value < 0.05
    })

df_comparisons = pd.DataFrame(comparisons)

print("\n  Legend: *** p<0.001, ** p<0.01, * p<0.05, n.s. = not significant")

# --- 6. 关键可视化 1: 模型性能对比（带置信区间的条形图）---
print("\n[Visualization 1/3] Generating performance comparison bar chart...")

fig, ax = plt.subplots(figsize=(12, 7))

models = df_benchmark['Model']
x_pos = np.arange(len(models))
colors = ['darkorange', 'green', 'purple', 'blue']

bars = ax.bar(x_pos, df_benchmark['AUC'], 
              yerr=[df_benchmark['AUC'] - df_benchmark['AUC_CI_Lower'],
                    df_benchmark['AUC_CI_Upper'] - df_benchmark['AUC']],
              color=colors, alpha=0.8, capsize=8, width=0.6,
              error_kw={'linewidth': 2, 'elinewidth': 2})

# 添加数值标签
for i, (model, auc_val, ci_lower, ci_upper) in enumerate(zip(
    models, df_benchmark['AUC'], df_benchmark['AUC_CI_Lower'], df_benchmark['AUC_CI_Upper'])):
    
    # 主要数值
    ax.text(i, auc_val + 0.01, f'{auc_val:.4f}',
            ha='center', va='bottom', fontsize=13, fontweight='bold')
    
    # 置信区间
    ax.text(i, ci_lower - 0.015, f'95% CI:\n[{ci_lower:.4f},\n{ci_upper:.4f}]',
            ha='center', va='top', fontsize=9, style='italic')

ax.set_ylabel('AUC Score', fontsize=14, fontweight='bold')
ax.set_xlabel('Classification Method', fontsize=14, fontweight='bold')
ax.set_title(f'Performance Comparison: ML Models vs Traditional Method\n'
             f'(Pion vs Proton, p={MOMENTUM_OVERLAP_MIN}-{MOMENTUM_OVERLAP_MAX} GeV/c)',
             fontsize=15, fontweight='bold', pad=20)
ax.set_xticks(x_pos)
ax.set_xticklabels(models, fontsize=12, fontweight='bold')
ax.set_ylim(0.75, 0.85)
ax.grid(True, alpha=0.3, axis='y', linestyle='--')
ax.axhline(y=0.8, color='gray', linestyle=':', linewidth=1.5, alpha=0.5)

# 添加统计显著性标记（仅标记最重要的对比）
# MLP vs Truncated Mean
mlp_idx, tm_idx = 0, 3
y_max = max(df_benchmark.loc[mlp_idx, 'AUC_CI_Upper'], 
            df_benchmark.loc[tm_idx, 'AUC_CI_Upper'])
p_val_mlp_tm = df_comparisons[df_comparisons['Comparison']=='MLP vs Truncated Mean']['p_value'].values[0]
sig_marker = "***" if p_val_mlp_tm < 0.001 else "**" if p_val_mlp_tm < 0.01 else "*"
ax.plot([mlp_idx, tm_idx], [y_max+0.015, y_max+0.015], 'k-', linewidth=1.5)
ax.text((mlp_idx+tm_idx)/2, y_max+0.018, sig_marker, ha='center', fontsize=16)

plt.tight_layout()
plt.savefig('ml_benchmark_comparison.png', dpi=300, bbox_inches='tight')
plt.show()
print("  ✓ Saved: ml_benchmark_comparison.png")

# --- 7. 关键可视化 2: ROC曲线对比（所有模型）---
print("\n[Visualization 2/3] Generating comprehensive ROC curves...")

fig, ax = plt.subplots(figsize=(10, 9))

# 计算所有模型的ROC曲线
fpr_xgb, tpr_xgb, _ = roc_curve(y_test, y_scores_xgb)
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_scores_rf)

# 绘制ROC曲线（从最差到最好，使truncated mean在底层）
models_roc = [
    ('Truncated Mean', fpr_truncated, tpr_truncated, auc_tm_mean, 'blue', '--', 2),
    ('Random Forest', fpr_rf, tpr_rf, auc_rf_mean, 'purple', '-.', 2.5),
    ('XGBoost', fpr_xgb, tpr_xgb, auc_xgb_mean, 'green', '-', 2.5),
    ('MLP', fpr_mlp, tpr_mlp, auc_mlp_mean, 'darkorange', '-', 3),
]

for name, fpr, tpr, auc_score, color, linestyle, linewidth in models_roc:
    ax.plot(fpr, tpr, color=color, linestyle=linestyle, linewidth=linewidth,
            label=f'{name} (AUC = {auc_score:.4f})')

# 对角线
ax.plot([0, 1], [0, 1], color='gray', linestyle=':', linewidth=2, alpha=0.6,
        label='Random Guess')

ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.set_xlabel('False Positive Rate', fontsize=13, fontweight='bold')
ax.set_ylabel('True Positive Rate', fontsize=13, fontweight='bold')
ax.set_title('ROC Curves: Machine Learning Models vs Traditional Method\n'
             f'(Pion vs Proton, p={MOMENTUM_OVERLAP_MIN}-{MOMENTUM_OVERLAP_MAX} GeV/c)',
             fontsize=14, fontweight='bold', pad=15)
ax.legend(loc="lower right", fontsize=11, framealpha=0.95)
ax.grid(True, alpha=0.3)

# 添加性能排名文本框
ranking_text = "Performance Ranking:\n"
sorted_models = df_benchmark.sort_values('AUC', ascending=False)
for i, row in enumerate(sorted_models.itertuples(), 1):
    improvement_vs_tm = ((row.AUC - auc_tm_mean) / auc_tm_mean * 100) if row.Model != 'Truncated Mean' else 0
    ranking_text += f"{i}. {row.Model}: {row.AUC:.4f}"
    if row.Model != 'Truncated Mean':
        ranking_text += f" (+{improvement_vs_tm:.1f}%)"
    ranking_text += "\n"

props = dict(boxstyle='round', facecolor='wheat', alpha=0.9, edgecolor='black', linewidth=1.5)
ax.text(0.98, 0.02, ranking_text.strip(), transform=ax.transAxes,
        fontsize=10, verticalalignment='bottom', horizontalalignment='right',
        bbox=props, family='monospace')

plt.tight_layout()
plt.savefig('ml_benchmark_roc_curves.png', dpi=300, bbox_inches='tight')
plt.show()
print("  ✓ Saved: ml_benchmark_roc_curves.png")

# --- 8. 关键可视化 3: Bootstrap AUC分布对比 ---
print("\n[Visualization 3/3] Generating Bootstrap AUC distribution comparison...")

fig, ax = plt.subplots(figsize=(12, 7))

# 绘制所有模型的Bootstrap分布
bootstrap_data = [
    (bootstrap_aucs_mlp, 'MLP', 'darkorange'),
    (bootstrap_aucs_xgb, 'XGBoost', 'green'),
    (bootstrap_aucs_rf, 'Random Forest', 'purple'),
    (bootstrap_aucs_tm, 'Truncated Mean', 'blue')
]

for aucs, name, color in bootstrap_data:
    ax.hist(aucs, bins=40, alpha=0.5, color=color, label=name, density=True)
    mean_val = np.mean(aucs)
    ax.axvline(mean_val, color=color, linestyle='--', linewidth=2.5,
               label=f'{name} Mean: {mean_val:.4f}')

ax.set_xlabel('AUC Score', fontsize=13, fontweight='bold')
ax.set_ylabel('Probability Density', fontsize=13, fontweight='bold')
ax.set_title('Bootstrap Distribution of AUC: All Models Comparison\n'
             '(1000 Bootstrap Samples)',
             fontsize=14, fontweight='bold', pad=15)
ax.legend(loc='upper left', fontsize=10, ncol=2)
ax.grid(True, alpha=0.3, linestyle='--')
ax.set_xlim(0.70, 0.82)

plt.tight_layout()
plt.savefig('ml_benchmark_bootstrap_distributions.png', dpi=300, bbox_inches='tight')
plt.show()
print("  ✓ Saved: ml_benchmark_bootstrap_distributions.png")

# --- 9. 生成详细的基准测试报告 ---
print("\n" + "="*80)
print("MACHINE LEARNING BENCHMARK REPORT")
print("="*80)

print(f"\nDataset: Pion vs Proton, Momentum Range: {MOMENTUM_OVERLAP_MIN}-{MOMENTUM_OVERLAP_MAX} GeV/c")
print(f"Training Samples: {len(X_train)}, Test Samples: {len(X_test)}")
print(f"Features: {X_train.shape[1]} (50 sorted dE/dx + 5 statistical features)")

print("\n" + "-"*80)
print("Model Performance Summary:")
print("-"*80)
print(f"{'Model':<18} {'AUC':<12} {'95% CI':<28} {'Accuracy':<10} {'Train Time':<12}")
print("-"*80)
for _, row in df_benchmark.iterrows():
    ci_str = f"[{row['AUC_CI_Lower']:.4f}, {row['AUC_CI_Upper']:.4f}]"
    time_str = f"{row['Training_Time_s']:.2f}s" if not np.isnan(row['Training_Time_s']) else "N/A"
    print(f"{row['Model']:<18} {row['AUC']:.4f}      {ci_str:<28} {row['Accuracy']:.4f}    {time_str:<12}")
print("-"*80)

print("\n" + "-"*80)
print("Statistical Significance (Pairwise Comparisons):")
print("-"*80)
print(f"{'Comparison':<40} {'ΔAUC':<10} {'p-value':<12} {'Effect Size (d)':<15} {'Significant?':<12}")
print("-"*80)
for _, row in df_comparisons.iterrows():
    sig_str = "YES" if row['Significant'] else "NO"
    print(f"{row['Comparison']:<40} {row['Mean_Difference']:+.4f}    {row['p_value']:.6f}   {row['Cohens_d']:+.3f}            {sig_str:<12}")
print("-"*80)

print("\n" + "="*80)
print("KEY FINDINGS:")
print("="*80)

# 找出最佳模型
best_model_idx = df_benchmark['AUC'].idxmax()
best_model = df_benchmark.loc[best_model_idx]

print(f"\n1. BEST PERFORMING MODEL: {best_model['Model']}")
print(f"   - AUC: {best_model['AUC']:.4f} [{best_model['AUC_CI_Lower']:.4f}, {best_model['AUC_CI_Upper']:.4f}]")
print(f"   - Accuracy: {best_model['Accuracy']:.4f}")

# 与传统方法对比
improvement_vs_tm = best_model['AUC'] - auc_tm_mean
rel_improvement = (improvement_vs_tm / auc_tm_mean) * 100
comparison_tm = df_comparisons[df_comparisons['Comparison'].str.contains(best_model['Model']) & 
                               df_comparisons['Comparison'].str.contains('Truncated Mean')].iloc[0]

print(f"\n2. IMPROVEMENT OVER TRADITIONAL METHOD:")
print(f"   - Absolute Improvement: ΔAUC = {improvement_vs_tm:+.4f}")
print(f"   - Relative Improvement: {rel_improvement:+.2f}%")
print(f"   - Statistical Significance: p = {comparison_tm['p_value']:.6f}")
print(f"   - Effect Size (Cohen's d): {comparison_tm['Cohens_d']:.3f}")
if comparison_tm['Significant']:
    print("   ✓ Improvement is STATISTICALLY SIGNIFICANT")

# ML模型之间的对比
print(f"\n3. COMPARISON AMONG ML MODELS:")
ml_models = df_benchmark[df_benchmark['Model'] != 'Truncated Mean']
auc_range = ml_models['AUC'].max() - ml_models['AUC'].min()
print(f"   - AUC Range: {auc_range:.4f}")

mlp_xgb_comp = df_comparisons[df_comparisons['Comparison'] == 'MLP vs XGBoost'].iloc[0]
mlp_rf_comp = df_comparisons[df_comparisons['Comparison'] == 'MLP vs Random Forest'].iloc[0]
xgb_rf_comp = df_comparisons[df_comparisons['Comparison'] == 'XGBoost vs Random Forest'].iloc[0]

print(f"   - MLP vs XGBoost: ΔAUC = {mlp_xgb_comp['Mean_Difference']:+.4f}, "
      f"p = {mlp_xgb_comp['p_value']:.4f} ({'Significant' if mlp_xgb_comp['Significant'] else 'Not Significant'})")
print(f"   - MLP vs Random Forest: ΔAUC = {mlp_rf_comp['Mean_Difference']:+.4f}, "
      f"p = {mlp_rf_comp['p_value']:.4f} ({'Significant' if mlp_rf_comp['Significant'] else 'Not Significant'})")
print(f"   - XGBoost vs Random Forest: ΔAUC = {xgb_rf_comp['Mean_Difference']:+.4f}, "
      f"p = {xgb_rf_comp['p_value']:.4f} ({'Significant' if xgb_rf_comp['Significant'] else 'Not Significant'})")

print("\n" + "="*80)
print("CONCLUSION:")
print("="*80)

# 判断ML模型之间是否有显著差异
ml_comparisons = df_comparisons[~df_comparisons['Comparison'].str.contains('Truncated Mean')]
any_ml_significant = ml_comparisons['Significant'].any()

if not any_ml_significant:
    print("✓ All three ML models (MLP, XGBoost, Random Forest) perform COMPARABLY well")
    print("  with no statistically significant differences between them.")
    print(f"  Average ML AUC: {ml_models['AUC'].mean():.4f} ± {ml_models['AUC'].std():.4f}")
elif best_model['Model'] == 'MLP':
    print("✓ The MLP model shows the best performance, though the advantage over")
    print("  other ML models may be modest.")
else:
    print(f"✓ {best_model['Model']} shows the best performance among tested models.")

print(f"\n✓ ALL machine learning approaches SIGNIFICANTLY OUTPERFORM")
print(f"  the traditional truncated mean method (average improvement: "
      f"{((ml_models['AUC'].mean() - auc_tm_mean) / auc_tm_mean * 100):.2f}%)")

print("\n" + "="*80)

# 保存详细结果
df_benchmark.to_csv('ml_benchmark_results.csv', index=False)
df_comparisons.to_csv('ml_benchmark_statistical_tests.csv', index=False)

print("\n✓ Benchmark analysis complete!")
print("  Generated files:")
print("  - ml_benchmark_comparison.png (关键图1: 性能对比)")
print("  - ml_benchmark_roc_curves.png (关键图2: ROC曲线)")
print("  - ml_benchmark_bootstrap_distributions.png (关键图3: 统计分布)")
print("  - ml_benchmark_results.csv")
print("  - ml_benchmark_statistical_tests.csv")
print("="*80)

# 2.1.3 系统性不确定性

In [ ]:
print("\n" + "="*80)
print("SYSTEMATIC UNCERTAINTY ANALYSIS")
print("="*80)

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, accuracy_score
import copy
import warnings
warnings.filterwarnings('ignore') # 忽略一些可能出现的计算警告

# 1. 定义不确定性场景
# TPC dE/dx 刻度不确定性：以百分比表示的系统性偏置
# 扩大测试范围：例如从 ±2% 扩展到 ±10%，并增加更多测试点
# dedx_calibration_uncertainties_pct = np.linspace(-0.02, 0.02, 5) # 旧范围
dedx_calibration_uncertainties_pct = np.array([-0.10, -0.05, -0.03, -0.02, -0.01, 0.0, 0.01, 0.02, 0.03, 0.05, 0.10]) # 新范围：-10%到+10%，共11个点

# 动量分辨率不确定性：以百分比表示的动量测量标准差 (σ_p / p)
# 扩大测试范围：例如从 0% 到 3% 扩展到 0% 到 10%
# momentum_resolution_uncertainties_pct = np.linspace(0.0, 0.03, 7) # 旧范围
momentum_resolution_uncertainties_pct = np.linspace(0.0, 0.10, 11) # 新范围：0%到10%，共11个点

print(f"\nDefining uncertainty scenarios:")
print(f"  dE/dx Calibration Uncertainties (pct): {[f'{p*100:.1f}%' for p in dedx_calibration_uncertainties_pct]}")
print(f"  Momentum Resolution Uncertainties (pct): {[f'{p*100:.1f}%' for p in momentum_resolution_uncertainties_pct]}")

# 确保 MLP 和 截断均值法的最佳模型和 scaler 已经加载
# best_model 是 MLP 分类器
# scaler_classifier 是 MLP 输入特征的 StandardScaler
# y_test 是真实标签
# y_scores_mlp 是 MLP 在原始测试集上的预测分数
# dedx_test 是截断均值在原始测试集上的值 (用于截断均值法的AUC计算)
# p_test 是原始测试集动量
# X_test 是原始测试集输入特征
# y_truncated_mean_full 是原始截断均值全量数据
# momentum_data_full 是原始动量全量数据
# mass_data_full 是原始质量全量数据
# X_data_full 是原始特征全量数据

# 重新加载MLP模型（如果模型在之前的步骤中被清除或修改）
# model_classifier = MLPClassifierOptimized(input_size, hidden_size_classifier).to(device)
# model_classifier.load_state_dict(torch.load('best_mlp_classifier_optimized.pth'))
# model_classifier.eval() # 设置为评估模式

# 2. 为每个不确定性场景评估模型性能

def evaluate_with_uncertainty(
    dedx_bias_factor, mom_res_sigma_p_pct,
    original_X_data_full, original_y_truncated_mean_full,
    original_mass_data_full, original_momentum_data_full,
    mlp_model, mlp_scaler, target_mass1, target_mass2,
    mom_overlap_min, mom_overlap_max,
    input_feat_num_dedx_values, # 50
    input_feat_start_idx_stats # 50 (索引从0开始)
):
    """
    在给定系统不确定性条件下，重新构建测试集并评估MLP和截断均值法的性能。
    """
    # 深度复制原始数据以避免修改
    current_dedx_data_full = original_y_truncated_mean_full * dedx_bias_factor
    current_momentum_data_full = original_momentum_data_full.copy()
    
    # 对原始特征 X_data_full 也应用 dE/dx 偏置
    current_X_data_full = original_X_data_full.copy()
    
    # 扰动 dE/dx 原始值 (前 NUM_DE_DX_VALUES 列)
    current_X_data_full[:, :input_feat_num_dedx_values] *= dedx_bias_factor
    
    # 扰动统计特征 (在 NUM_DE_DX_VALUES 之后的列)，假设它们是从扰动后的原始值计算得来
    # 简单地对 Mean 和 Median 应用相同因子，Std Dev, Skewness, Kurtosis可能需要更复杂的重新计算
    # 但为了简化模拟，我们只对与均值和中位数相关的统计特征进行调整。
    # 这是一个简化，实际情况可能更复杂。
    # 这里需要根据你的特征顺序调整，假设 Mean是第50列, Median是第52列
    # 确认你的特征顺序，在你的代码中 X_data_full 的特征结构是：
    # [dE/dx_val_1, ..., dE/dx_val_50, Mean, StdDev, Median, Skewness, Kurtosis]
    # 所以 Mean 是第 50 个特征 (索引 50), Median 是第 52 个特征 (索引 52)
    # 请根据实际特征索引调整
    current_X_data_full[:, input_feat_start_idx_stats] *= dedx_bias_factor # Mean
    # current_X_data_full[:, input_feat_start_idx_stats + 1] # StdDev 通常不直接缩放
    current_X_data_full[:, input_feat_start_idx_stats + 2] *= dedx_bias_factor # Median

    # 动量分辨率不确定性：在动量数据上添加高斯噪声
    if mom_res_sigma_p_pct > 0:
        # sigma_p = p * (sigma_p / p)
        # 注意：这里噪声的标准差是基于每个样本的动量值计算的，是乘性噪声。
        momentum_noise_std = current_momentum_data_full * mom_res_sigma_p_pct
        momentum_noise = np.random.normal(0, momentum_noise_std)
        current_momentum_data_full += momentum_noise
        # 确保动量不为负
        current_momentum_data_full = np.maximum(current_momentum_data_full, 0.01)

    # 重新筛选用于分类的样本（Pion vs Proton, 在重叠动量区域）
    # 注意：这里的 mask 使用的是原始粒子的质量，但动量使用的是扰动后的动量
    mask_pions = (original_mass_data_full == target_mass1) & \
                 (current_momentum_data_full >= mom_overlap_min) & \
                 (current_momentum_data_full < mom_overlap_max)
    mask_protons = (original_mass_data_full == target_mass2) & \
                   (current_momentum_data_full >= mom_overlap_min) & \
                   (current_momentum_data_full < mom_overlap_max)

    X_pions_perturbed = current_X_data_full[mask_pions]
    X_protons_perturbed = current_X_data_full[mask_protons]
    dedx_pions_perturbed = current_dedx_data_full[mask_pions]
    dedx_protons_perturbed = current_dedx_data_full[mask_protons]
    
    n_pions_perturbed = len(X_pions_perturbed)
    n_protons_perturbed = len(X_protons_perturbed)

    if n_pions_perturbed < 100 or n_protons_perturbed < 100: # 确保有足够的数据进行评估
        # print(f"  Warning: Insufficient data for this scenario: Pions={n_pions_perturbed}, Protons={n_protons_perturbed}")
        return np.nan, np.nan # 返回NaN表示无法评估

    X_test_perturbed = np.vstack((X_pions_perturbed, X_protons_perturbed))
    y_test_perturbed = np.hstack((np.zeros(n_pions_perturbed), np.ones(n_protons_perturbed)))
    dedx_test_perturbed = np.hstack((dedx_pions_perturbed, dedx_protons_perturbed))

    # MLP 评估 (使用原始训练时的 scaler)
    X_test_scaled_perturbed = mlp_scaler.transform(X_test_perturbed)
    X_test_tensor_perturbed = torch.tensor(X_test_scaled_perturbed, dtype=torch.float32).to(device)

    with torch.no_grad():
        y_scores_mlp_perturbed = mlp_model(X_test_tensor_perturbed).cpu().numpy().flatten()
    
    auc_mlp_perturbed = roc_auc_score(y_test_perturbed, y_scores_mlp_perturbed)
    
    # 截断均值法评估
    # 注意：截断均值法的 AUC 通常通过其原始值作为分数来计算，不需要重新计算阈值
    auc_tm_perturbed = roc_auc_score(y_test_perturbed, dedx_test_perturbed)
    
    return auc_mlp_perturbed, auc_tm_perturbed

# 存储结果
mlp_auc_dedx_calib = []
tm_auc_dedx_calib = []

mlp_auc_mom_res = []
tm_auc_mom_res = []

# ==================== dE/dx 刻度不确定性分析 ====================
print("\nPerforming dE/dx calibration uncertainty analysis...")
for bias_pct in dedx_calibration_uncertainties_pct:
    bias_factor = 1 + bias_pct
    print(f"  Evaluating with dE/dx bias: {bias_pct*100:.1f}%")
    
    auc_mlp, auc_tm = evaluate_with_uncertainty(
        dedx_bias_factor=bias_factor,
        mom_res_sigma_p_pct=0, # 动量分辨率设为0
        original_X_data_full=X_data_full,
        original_y_truncated_mean_full=y_truncated_mean_full,
        original_mass_data_full=mass_data_full,
        original_momentum_data_full=momentum_data_full,
        mlp_model=model_classifier,
        mlp_scaler=scaler_classifier,
        target_mass1=TARGET_MASS_1,
        target_mass2=TARGET_MASS_2,
        mom_overlap_min=MOMENTUM_OVERLAP_MIN,
        mom_overlap_max=MOMENTUM_OVERLAP_MAX,
        input_feat_num_dedx_values=NUM_DE_DX_VALUES,
        input_feat_start_idx_stats=NUM_DE_DX_VALUES
    )
    mlp_auc_dedx_calib.append(auc_mlp)
    tm_auc_dedx_calib.append(auc_tm)
    print(f"    MLP AUC: {auc_mlp:.4f}, TruncMean AUC: {auc_tm:.4f}")

# ==================== 动量分辨率不确定性分析 ====================
print("\nPerforming momentum resolution uncertainty analysis...")
for res_pct in momentum_resolution_uncertainties_pct:
    print(f"  Evaluating with momentum resolution (σ_p/p): {res_pct*100:.1f}%")
    
    # 为动量分辨率分析时，dE/dx 刻度设为1 (无偏置)
    auc_mlp, auc_tm = evaluate_with_uncertainty(
        dedx_bias_factor=1,
        mom_res_sigma_p_pct=res_pct,
        original_X_data_full=X_data_full,
        original_y_truncated_mean_full=y_truncated_mean_full,
        original_mass_data_full=mass_data_full,
        original_momentum_data_full=momentum_data_full,
        mlp_model=model_classifier,
        mlp_scaler=scaler_classifier,
        target_mass1=TARGET_MASS_1,
        target_mass2=TARGET_MASS_2,
        mom_overlap_min=MOMENTUM_OVERLAP_MIN,
        mom_overlap_max=MOMENTUM_OVERLAP_MAX,
        input_feat_num_dedx_values=NUM_DE_DX_VALUES,
        input_feat_start_idx_stats=NUM_DE_DX_VALUES
    )
    mlp_auc_mom_res.append(auc_mlp)
    tm_auc_mom_res.append(auc_tm)
    print(f"    MLP AUC: {auc_mlp:.4f}, TruncMean AUC: {auc_tm:.4f}")

# 3. 可视化不确定性影响 (关键图)
print("\nGenerating uncertainty plots...")

# a) dE/dx 刻度不确定性影响
plt.figure(figsize=(10, 7))
plt.plot(dedx_calibration_uncertainties_pct * 100, mlp_auc_dedx_calib, 
         'o-', color='darkorange', label='MLP Classifier AUC', lw=2)
plt.plot(dedx_calibration_uncertainties_pct * 100, tm_auc_dedx_calib, 
         's--', color='blue', label='Truncated Mean AUC', lw=2)

plt.xlabel('dE/dx Calibration Uncertainty (%)', fontsize=12)
plt.ylabel('AUC Score', fontsize=12)
plt.title('Impact of dE/dx Calibration Uncertainty on Particle Identification', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, linestyle='--', alpha=0.6)
# 调整Y轴以更好地显示可能的变化，范围可以根据实际结果调整
plt.ylim(min(0.65, np.nanmin(mlp_auc_dedx_calib), np.nanmin(tm_auc_dedx_calib) - 0.05), 
         max(0.85, np.nanmax(mlp_auc_dedx_calib), np.nanmax(tm_auc_dedx_calib) + 0.01)) 
plt.tight_layout()
plt.savefig('dedx_calibration_uncertainty_impact.png', dpi=300)
plt.show()

# b) 动量分辨率不确定性影响
plt.figure(figsize=(10, 7))
plt.plot(momentum_resolution_uncertainties_pct * 100, mlp_auc_mom_res, 
         'o-', color='darkorange', label='MLP Classifier AUC', lw=2)
plt.plot(momentum_resolution_uncertainties_pct * 100, tm_auc_mom_res, 
         's--', color='blue', label='Truncated Mean AUC', lw=2)

plt.xlabel('Momentum Resolution (σ_p/p) (%)', fontsize=12)
plt.ylabel('AUC Score', fontsize=12)
plt.title('Impact of Momentum Resolution Uncertainty on Particle Identification', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, linestyle='--', alpha=0.6)
# 调整Y轴以更好地显示可能的变化，范围可以根据实际结果调整
plt.ylim(min(0.65, np.nanmin(mlp_auc_mom_res), np.nanmin(tm_auc_mom_res) - 0.05), 
         max(0.85, np.nanmax(mlp_auc_mom_res), np.nanmax(tm_auc_mom_res) + 0.01)) 
plt.tight_layout()
plt.savefig('momentum_resolution_uncertainty_impact.png', dpi=300)
plt.show()

# 4. 讨论分析结果
print("\n" + "="*80)
print("SYSTEMATIC UNCERTAINTY ANALYSIS REPORT")
print("="*80)

print(f"\nAnalysis Parameters:")
print(f"  Particle Pair: {TARGET_MASS_1} (Pion) vs {TARGET_MASS_2} (Proton)")
print(f"  Momentum Range: {MOMENTUM_OVERLAP_MIN}-{MOMENTUM_OVERLAP_MAX} GeV/c")
print(f"  dE/dx Calibration Uncertainty Tested: {dedx_calibration_uncertainties_pct[0]*100:.1f}% to {dedx_calibration_uncertainties_pct[-1]*100:.1f}%")
print(f"  Momentum Resolution Uncertainty Tested: {momentum_resolution_uncertainties_pct[0]*100:.1f}% to {momentum_resolution_uncertainties_pct[-1]*100:.1f}%")

print("\nImpact of dE/dx Calibration Uncertainty:")
print("-" * 50)
# 过滤掉 NaN 值进行计算
valid_mlp_auc_dedx = [auc for auc in mlp_auc_dedx_calib if not np.isnan(auc)]
valid_tm_auc_dedx = [auc for auc in tm_auc_dedx_calib if not np.isnan(auc)]

if len(valid_mlp_auc_dedx) > 1 and len(valid_tm_auc_dedx) > 1:
    mlp_sensitivity_dedx = (valid_mlp_auc_dedx[-1] - valid_mlp_auc_dedx[0]) / (dedx_calibration_uncertainties_pct[len(dedx_calibration_uncertainties_pct)-1] - dedx_calibration_uncertainties_pct[0])
    tm_sensitivity_dedx = (valid_tm_auc_dedx[-1] - valid_tm_auc_dedx[0]) / (dedx_calibration_uncertainties_pct[len(dedx_calibration_uncertainties_pct)-1] - dedx_calibration_uncertainties_pct[0])
    
    print(f"  MLP AUC range: [{np.min(valid_mlp_auc_dedx):.4f}, {np.max(valid_mlp_auc_dedx):.4f}]")
    print(f"  Truncated Mean AUC range: [{np.min(valid_tm_auc_dedx):.4f}, {np.max(valid_tm_auc_dedx):.4f}]")
    print(f"  MLP Avg. Sensitivity to dE/dx bias: {mlp_sensitivity_dedx:.3f} AUC/%bias")
    print(f"  Truncated Mean Avg. Sensitivity to dE/dx bias: {tm_sensitivity_dedx:.3f} AUC/%bias")
    if np.abs(mlp_sensitivity_dedx) < np.abs(tm_sensitivity_dedx):
        print(f"  MLP is LESS sensitive to dE/dx calibration uncertainty.")
    elif np.abs(mlp_sensitivity_dedx) > np.abs(tm_sensitivity_dedx):
        print(f"  Truncated Mean is LESS sensitive to dE/dx calibration uncertainty.")
    else:
        print(f"  MLP and Truncated Mean show similar sensitivity to dE/dx calibration uncertainty.")
else:
    print("  Not enough valid data points to assess dE/dx calibration sensitivity.")


print("\nImpact of Momentum Resolution Uncertainty:")
print("-" * 50)
valid_mlp_auc_mom = [auc for auc in mlp_auc_mom_res if not np.isnan(auc)]
valid_tm_auc_mom = [auc for auc in tm_auc_mom_res if not np.isnan(auc)]

if len(valid_mlp_auc_mom) > 1 and len(valid_tm_auc_mom) > 1:
    mlp_sensitivity_mom_res = (valid_mlp_auc_mom[-1] - valid_mlp_auc_mom[0]) / (momentum_resolution_uncertainties_pct[len(momentum_resolution_uncertainties_pct)-1] - momentum_resolution_uncertainties_pct[0])
    tm_sensitivity_mom_res = (valid_tm_auc_mom[-1] - valid_tm_auc_mom[0]) / (momentum_resolution_uncertainties_pct[len(momentum_resolution_uncertainties_pct)-1] - momentum_resolution_uncertainties_pct[0])
    
    print(f"  MLP AUC range: [{np.min(valid_mlp_auc_mom):.4f}, {np.max(valid_mlp_auc_mom):.4f}]")
    print(f"  Truncated Mean AUC range: [{np.min(valid_tm_auc_mom):.4f}, {np.max(valid_tm_auc_mom):.4f}]")
    print(f"  MLP Avg. Sensitivity to momentum resolution: {mlp_sensitivity_mom_res:.3f} AUC/%res")
    print(f"  Truncated Mean Avg. Sensitivity to momentum resolution: {tm_sensitivity_mom_res:.3f} AUC/%res")
    if np.abs(mlp_sensitivity_mom_res) < np.abs(tm_sensitivity_mom_res):
        print(f"  MLP is LESS sensitive to momentum resolution uncertainty.")
    elif np.abs(mlp_sensitivity_mom_res) > np.abs(tm_sensitivity_mom_res):
        print(f"  Truncated Mean is LESS sensitive to momentum resolution uncertainty.")
    else:
        print(f"  MLP and Truncated Mean show similar sensitivity to momentum resolution uncertainty.")
else:
    print("  Not enough valid data points to assess momentum resolution sensitivity.")

print("\nSummary:")
print("-" * 80)
print("  This analysis highlights the robustness of both methods against common experimental uncertainties.")
print("  For dE/dx calibration, both methods generally show some dependency on the calibration quality.")
print("  For momentum resolution, increased uncertainty degrades performance for both, as expected.")
print("  The comparative sensitivity indicates which method might be more reliable under specific experimental conditions.")
print("  Further investigations would involve combining multiple uncertainties and exploring non-linear effects.")
print("="*80)

print("\n✓ Systematic uncertainty analysis complete!")
print("  Generated plots:")
print("  - dedx_calibration_uncertainty_impact.png")
print("  - momentum_resolution_uncertainty_impact.png")
print("="*80)